# 0.0 Imports

In [3]:
import pandas as pd
import numpy as np
import inflection
import umap as umap
import re
import s3fs

from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn import decomposition as dd
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestRegressor


## 0.1 Helper Functions

## 0.2 Load Data

In [5]:
path_s3 = 's3://insiders-dataset-nm'
data = pd.read_excel( path_s3 + '/Online Retail.xlsx/')

PermissionError: Forbidden

# 1.0 Data Description

### 1.2 Rename Columns

In [3]:
cols_old = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country']

snakecase = lambda x: inflection.underscore( x ) # change to snakecase
cols_new = list( map( snakecase, cols_old ))

data.columns = cols_new # define the new columns

### 1.3 Check and fillout Nan

In [4]:
# replace NA
df_missing = data[data['customer_id'].isna()]
df_not_missing = data[~data['customer_id'].isna()]

In [5]:
# create reference
backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
backup['customer_id'] = np.arange(19000, 19000+len(backup), 1)

# merge original data frame
df1 = pd.merge(data, backup, on='invoice_no', how='left')

#coalesce
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])

# drop columns
df1 = df1.drop(columns=['customer_id_x', 'customer_id_y'], axis=1)

### 1.4 Change dtypes

In [6]:
# invoice data to datetime
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%d-%b-%y')
#df1['invoice_date'] = df1['invoice_date'].dt.date

# customer id
df1['customer_id'] = df1['customer_id'].astype(int)

# 2.0 Filtragem de variáveis

In [7]:
df2 = df1.copy()

In [8]:
# unit price > 0.04
df2 = df2.loc[df2['unit_price'] >= 0.04, :]

# stock code 
df2 = df2[~df2['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL','PADS', 'B', 'CRUK'])]

# description
df2 = df2.drop(columns='description', axis=1)

#map
df2 = df2[~df2['country'].isin(['European Community', 'Unspecified'])]

# bad users
df2 = df2[~df2['customer_id'].isin([16446])]

# quantity - negative numbers means product returns
df2_returns = df2.loc[df2['quantity'] < 0, :]
df2_purchases = df2.loc[df2['quantity'] >= 0, :]

# 3.0 Feature Engineering

In [9]:
# date reference 
df_ref = df2_purchases.drop(['invoice_no', 'stock_code',
                   'quantity', 'invoice_date', 'unit_price', 
                   'country'], axis=1).drop_duplicates(ignore_index=True)

In [10]:
df_ref.shape

(5695, 1)

### 3.1 Features

In [11]:
# Gross Revenue ( Faturamento ) quantity * price
df2_purchases.loc[:, 'gross_revenue'] = df2_purchases.loc[:, 'quantity'] * df2_purchases.loc[:, 'unit_price']

# Monetary
df_monetary = df2_purchases.loc[:, ['customer_id', 'gross_revenue']].groupby( 'customer_id' ).sum().reset_index()
df_ref = pd.merge( df_ref, df_monetary, on='customer_id', how='left' )
df_ref.isna().sum()

/tmp/ipykernel_69667/206990356.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_purchases.loc[:, 'gross_revenue'] = df2_purchases.loc[:, 'quantity'] * df2_purchases.loc[:, 'unit_price']


customer_id      0
gross_revenue    0
dtype: int64

In [12]:
# Recency - Last day purchase
df_recency = df2_purchases.loc[:, ['customer_id', 'invoice_date']].groupby( 'customer_id' ).max().reset_index()
df_recency['recency_days'] = ( df2['invoice_date'].max() - df_recency['invoice_date'] ).dt.days
df_recency['recency_days'] = df_recency['recency_days'].apply( lambda x: 1/x if x !=0 else 1)
df_recency = df_recency[['customer_id', 'recency_days']].copy()
df_ref = pd.merge( df_ref, df_recency, on='customer_id', how='left' )
df_ref.isna().sum()

customer_id      0
gross_revenue    0
recency_days     0
dtype: int64

In [13]:
# qtde_invoices
df_freq = (df2_purchases.loc[:, ['customer_id', 'invoice_no']].drop_duplicates()
                                                             .groupby( 'customer_id' )
                                                             .count()
                                                             .reset_index()
                                                             .rename( columns={'invoice_no': 'qtde_invoices'}) )
df_ref = pd.merge( df_ref, df_freq, on='customer_id', how='left' )
df_ref.isna().sum()

customer_id      0
gross_revenue    0
recency_days     0
qtde_invoices    0
dtype: int64

In [14]:
# qtde_items
df_freq = (df2_purchases.loc[:, ['customer_id', 'quantity']].groupby( 'customer_id' ).sum()
                                                           .reset_index()
                                                           .rename( columns={'quantity': 'qtde_items'} ) )
df_ref = pd.merge( df_ref, df_freq, on='customer_id', how='left' )
df_ref.isna().sum()

customer_id      0
gross_revenue    0
recency_days     0
qtde_invoices    0
qtde_items       0
dtype: int64

In [15]:
# Numero de produtos
df_freq = (df2_purchases.loc[:, ['customer_id', 'stock_code']].groupby( 'customer_id' ).count()
                                                           .reset_index()
                                                           .rename( columns={'stock_code': 'qtde_products'} ) )
df_ref = pd.merge( df_ref, df_freq, on='customer_id', how='left' )
df_ref.isna().sum()

customer_id      0
gross_revenue    0
recency_days     0
qtde_invoices    0
qtde_items       0
qtde_products    0
dtype: int64

In [16]:
# Avg Ticket
df_avg_ticket = df2_purchases.loc[:, ['customer_id', 'gross_revenue']].groupby( 'customer_id' ).mean().reset_index().rename( columns={'gross_revenue':'avg_ticket'} )
df_ref = pd.merge( df_ref, df_avg_ticket, on='customer_id', how='left')
df_ref.isna().sum()

customer_id      0
gross_revenue    0
recency_days     0
qtde_invoices    0
qtde_items       0
qtde_products    0
avg_ticket       0
dtype: int64

In [17]:
# Average recency days
df_aux = df2[['customer_id', 'invoice_date']].drop_duplicates().sort_values( ['customer_id', 'invoice_date'], ascending= False )
df_aux['next_customer_id'] = df_aux['customer_id'].shift() # next customer
df_aux['previous_date'] = df_aux['invoice_date'].shift() # next invoince date

df_aux['avg_recency_days'] = df_aux.apply( lambda x: ( x['previous_date'] - x['invoice_date'] ).days if x['customer_id'] == x['next_customer_id'] else 0, axis=1 )
df_aux['avg_recency_days'] = -1* df_aux['avg_recency_days'].apply( lambda x: 1/x if x!=0 else 1)

df_aux = df_aux.drop( ['invoice_date', 'next_customer_id', 'previous_date'], axis=1 ).dropna()

# average recency 
df_avg_recency_days = df_aux.groupby( 'customer_id' ).mean().reset_index()

# merge
df_ref = pd.merge( df_ref, df_avg_recency_days, on='customer_id', how='left' )
df_ref.isna().sum()

customer_id         0
gross_revenue       0
recency_days        0
qtde_invoices       0
qtde_items          0
qtde_products       0
avg_ticket          0
avg_recency_days    0
dtype: int64

In [18]:
# frquency
df_aux = ( df2_purchases[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                             .groupby( 'customer_id')
                                                             .agg( max_ = ( 'invoice_date', 'max' ), 
                                                                   min_ = ( 'invoice_date', 'min' ),
                                                                   days_= ( 'invoice_date', lambda x: ( ( x.max() - x.min() ).days ) + 1 ),
                                                                   buy_ = ( 'invoice_no', 'count' ) ) ).reset_index()
# Frequency
df_aux['frequency'] = df_aux[['buy_', 'days_']].apply( lambda x: x['buy_'] / x['days_'] if  x['days_'] != 0 else 0, axis=1 )

# Merge
df_ref = pd.merge( df_ref, df_aux[['customer_id', 'frequency']], on='customer_id', how='left' )

df_ref.isna().sum()

customer_id         0
gross_revenue       0
recency_days        0
qtde_invoices       0
qtde_items          0
qtde_products       0
avg_ticket          0
avg_recency_days    0
frequency           0
dtype: int64

In [19]:
# Number of Returns
df_returns = df2_returns[['customer_id', 'quantity']].groupby( 'customer_id' ).sum().reset_index().rename( columns={'quantity':'qtde_returns'} )
df_returns['qtde_returns'] = df_returns['qtde_returns'] * -1
df_returns['qtde_returns'] = df_returns['qtde_returns'].apply( lambda x: 1/x if x!= 0 else 0)


df_ref = pd.merge( df_ref, df_returns, how='left', on='customer_id' )
df_ref.loc[df_ref['qtde_returns'].isna(), 'qtde_returns'] = 0

df_ref.isna().sum()

customer_id         0
gross_revenue       0
recency_days        0
qtde_invoices       0
qtde_items          0
qtde_products       0
avg_ticket          0
avg_recency_days    0
frequency           0
qtde_returns        0
dtype: int64

In [20]:
# avg_basket_size
df_aux = ( df2_purchases.loc[:, ['customer_id', 'invoice_no', 'quantity']].groupby( 'customer_id' )
                                                                            .agg( n_purchase=( 'invoice_no', 'nunique'),
                                                                                  n_products=( 'quantity', 'sum' ) )
                                                                            .reset_index() )

# calculation
df_aux['avg_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

# merge
df_ref = pd.merge( df_ref, df_aux[['customer_id', 'avg_basket_size']], how='left', on='customer_id' )
df_ref.isna().sum()

customer_id         0
gross_revenue       0
recency_days        0
qtde_invoices       0
qtde_items          0
qtde_products       0
avg_ticket          0
avg_recency_days    0
frequency           0
qtde_returns        0
avg_basket_size     0
dtype: int64

In [21]:
# avg_unique_basket_size
df_aux = ( df2_purchases.loc[:, ['customer_id', 'invoice_no', 'stock_code']].groupby( 'customer_id' )
                                                                            .agg( n_purchase=( 'invoice_no', 'nunique'),
                                                                                   n_products=( 'stock_code', 'nunique' ) )
                                                                            .reset_index() )

# calculation
df_aux['avg_unique_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

# merge
df_ref = pd.merge( df_ref, df_aux[['customer_id', 'avg_unique_basket_size']], how='left', on='customer_id' )
df_ref.isna().sum()

customer_id               0
gross_revenue             0
recency_days              0
qtde_invoices             0
qtde_items                0
qtde_products             0
avg_ticket                0
avg_recency_days          0
frequency                 0
qtde_returns              0
avg_basket_size           0
avg_unique_basket_size    0
dtype: int64

In [22]:
# money returned
customers_return = df2[df2['quantity']< 0]
customers_return.loc[:, 'revenue_returned'] = customers_return.loc[:, 'quantity'] * customers_return.loc[:, 'unit_price']
revenue_returned = customers_return.groupby('customer_id')['revenue_returned'].sum().reset_index()
revenue_returned['revenue_returned'] = revenue_returned['revenue_returned']

df_ref = pd.merge(df_ref, revenue_returned, on='customer_id', how='left')
df_ref = df_ref.fillna(0)

/tmp/ipykernel_69667/428756391.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers_return.loc[:, 'revenue_returned'] = customers_return.loc[:, 'quantity'] * customers_return.loc[:, 'unit_price']


# 4.0 EDA

In [23]:
df_ref = df_ref.dropna()
df4 = df_ref.copy()

### 4.3 Estudo de Espaço

In [24]:
df4_no_scale = df4.drop( columns=['customer_id'], axis=1).copy()

#### 4.4.4 Tree-Based Embedding

In [25]:
X = df4_no_scale.drop( columns=['gross_revenue'], axis=1 )
y = df4_no_scale['gross_revenue']

# model definition
rf = RandomForestRegressor( n_estimators=100,
                           criterion='friedman_mse',
                           random_state=42)
# model training
rf.fit(X, y)

# dataframe leaf
df_leaf = pd.DataFrame( rf.apply( X ) )

In [26]:
reducer = umap.UMAP( n_neighbors=20, random_state=42,  )
embedding = reducer.fit_transform( df_leaf )

# embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:, 0]
df_tree['embedding_y'] = embedding[:, 1]
# df_tree['embedding_z'] = embedding[:, 2]

# 5.0 Hyperparameter Fine-Tunning

In [27]:
df7 = df_tree.copy()
X = df7.copy()

# 6.0 Machine Learnign Modeling

In [28]:
k = 11

In [29]:
# model definition
kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)

# model trainig 
kmeans.fit ( X )

# clustering
labels = kmeans.labels_

### Cluster Validation

In [30]:
# WSS 
print( f'WSS value: {kmeans.inertia_}')

# Silhouette
print(f'SS Value: {metrics.silhouette_score( X, labels, metric="euclidean")}')

WSS value: 35485.94140625
SS Value: 0.5762630105018616


# 7.0 Cluster Analysis

In [31]:
df9 = X.copy()
df9['cluster'] = labels

### Cluster Profile

In [32]:
df4['cluster'] = labels
df9 = df4.copy()

In [33]:
df9['recency_days'] = df9['recency_days'].apply( lambda x: 1/ x if x != 0 else 0)
df9['qtde_returns'] = df9['qtde_returns'].apply( lambda x: 1/ x if x != 0 else 0)
df9['avg_recency_days'] = df9['avg_recency_days'].apply( lambda x: 1/ x if x != 0 else 1)

In [34]:
# Number of customers
df_cluster = df9[['customer_id', 'cluster']].groupby('cluster').count().reset_index()
df_cluster['per_customer'] = 100 * df_cluster['customer_id'] / df_cluster['customer_id'].sum()

# Avg revenue
df_avg_revenue = df9[['gross_revenue', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_revenue, on='cluster', how='inner')

# Avg recency
df_avg_recency = df9[['recency_days', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_recency, on='cluster', how='inner')

# purchases
df_avg_frequency = df9[['qtde_invoices', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_frequency, on='cluster', how='inner')

# # Avg Ticket
# df_avg_ticket = df9[['revenue', 'cluster']].groupby('cluster').mean().reset_index()
# df_cluster = pd.merge(df_cluster, df_avg_ticket, on='cluster', how='inner')

# returns
df_avg_ticket = df9[['qtde_returns', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_ticket, on='cluster', how='inner')

# product_quantity
df_avg_ticket = df9[['qtde_products', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_ticket, on='cluster', how='inner')

# avg_frquency
df_avg_ticket = df9[['frequency', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_ticket, on='cluster', how='inner')

# quantity
df_avg_ticket = df9[['avg_basket_size', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_ticket, on='cluster', how='inner')

# AVG revenue returned
df_avg_money_returned = df9.groupby('cluster')['revenue_returned'].mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_money_returned, on='cluster', how='inner')

In [35]:
df_cluster.sort_values('gross_revenue', ascending=False)

,cluster,customer_id,per_customer,gross_revenue,recency_days,qtde_invoices,qtde_returns,qtde_products,frequency,avg_basket_size,revenue_returned
6,6,433,7.603161,12091.152102,30.050808,15.600462,336.785219,370.685912,0.138669,886.855937,-521.349607
9,9,216,3.792801,3686.197454,119.486111,3.305556,10.180556,313.361111,0.713998,900.065978,-25.495880
0,0,378,6.637401,2559.131958,40.230159,6.642857,21.944444,158.732804,0.099242,329.378138,-45.562778
3,3,809,14.205443,1602.229370,64.672435,4.074166,13.105068,109.128554,0.241996,319.153840,-29.735859
4,4,374,6.567164,1176.347727,117.334225,2.467914,5.026738,114.799465,0.561089,289.690618,-13.897299
7,7,564,9.903424,932.056915,99.930851,2.434397,7.420213,66.781915,0.386474,244.434752,-15.991064
1,1,376,6.602283,548.072021,97.321809,2.210106,3.678191,42.821809,0.426589,186.620966,-10.138697
8,8,794,13.942054,442.860227,136.267003,1.714106,2.068010,41.696474,0.650830,128.424160,-7.282380
10,10,822,14.433714,259.274185,162.371046,1.238443,1.709246,16.588808,0.828490,108.228994,-5.005401
2,2,443,7.778753,143.865305,185.076749,1.130926,0.801354,13.550790,0.953968,36.414167,-2.852867


# 10.0 Deploy

In [36]:
df9.shape

(5695, 14)

In [37]:
from sqlalchemy import create_engine, text

# Database connection details
host = 'cluter-db.cx0iy0e6kyf8.sa-east-1.rds.amazonaws.com'
port = '5432'
database = 'postgres'
user = 'natan'
pwd = 'natan2019'

# Connection string
endpoint = f'postgresql://{user}:{pwd}@{host}:{port}/{database}'
conn = create_engine( endpoint )

# SQL query to create the table
query_create_table_insiders = """
    CREATE TABLE insiders (
        customer_id             INTEGER,
        gross_revenue           REAL,
        recency_days            REAL,
        qtde_invoices           INTEGER,
        qtde_items              INTEGER,
        qtde_products           INTEGER,
        avg_ticket              REAL,
        avg_recency_days        REAL,
        frequency               REAL,
        qtde_returns            REAL,
        avg_basket_size         REAL,
        avg_unique_basket_size  REAL,
        revenue_returned        REAL,
        cluster                 INTEGER
    )
"""

# Execute the query as raw SQL
with conn.connect() as conn:
    conn.execute(text(query_create_table_insiders))

# Assuming df9 is your DataFrame
df9.to_sql('insiders', con=conn, if_exists='append', index=False)
conn.close()
